# Using Python to Access Microsoft SQL

In [1]:
# import libraries
import pyodbc as py
import pandas as pd

In [2]:
# I need to find out the correct ODBC Driver Version
# I will loop through all the drivers I have access to
for driver in py.drivers():
    print(driver)

SQL Server
ODBC Driver 13 for SQL Server
SQL Server Native Client 11.0
SQL Server Native Client RDA 11.0
ODBC Driver 17 for SQL Server


In [3]:
# I have access to two versions, ODBC Driver 13 for SQL Server and ODBC Driver 17 for SQL Server
# I will use the latest one for the connection

In [4]:
# define the connection string
cnxn = py.connect("DRIVER={ODBC Driver 17 for SQL Server};"
                  "SERVER=DESKTOP-1UTCB6C;"
                  "DATABASE=AdventureWorks2017;"
                  "TRUSTED_Connection=yes;")

In [5]:
# Similar to example1, I will create a data frame using the table from the SQL Database
vendor = pd.read_sql_query('select * from Purchasing.PurchaseOrderHeader', cnxn)

In [6]:
# Confirmation that dtata frame 'vendor' is working
vendor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4012 entries, 0 to 4011
Data columns (total 13 columns):
PurchaseOrderID    4012 non-null int64
RevisionNumber     4012 non-null int64
Status             4012 non-null int64
EmployeeID         4012 non-null int64
VendorID           4012 non-null int64
ShipMethodID       4012 non-null int64
OrderDate          4012 non-null datetime64[ns]
ShipDate           4012 non-null datetime64[ns]
SubTotal           4012 non-null float64
TaxAmt             4012 non-null float64
Freight            4012 non-null float64
TotalDue           4012 non-null float64
ModifiedDate       4012 non-null datetime64[ns]
dtypes: datetime64[ns](3), float64(4), int64(6)
memory usage: 407.5 KB


In [7]:
# Quick preview of the data frame
vendor.head()

,PurchaseOrderID,RevisionNumber,Status,EmployeeID,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate
0,1,4,4,258,1580,3,2011-04-16,2011-04-25,201.0400,16.0832,5.0260,222.1492,2011-04-25
1,2,4,1,254,1496,5,2011-04-16,2011-04-25,272.1015,21.7681,6.8025,300.6721,2011-04-25
2,3,4,4,257,1494,2,2011-04-16,2011-04-25,8847.3000,707.7840,221.1825,9776.2665,2011-04-25
3,4,4,3,261,1650,5,2011-04-16,2011-04-25,171.0765,13.6861,4.2769,189.0395,2011-04-25
4,5,4,4,251,1654,4,2011-04-30,2011-05-09,20397.3000,1631.7840,509.9325,22539.0165,2011-05-09


In [8]:
# Now I will create a data frame from the bank transactions
tranx = pd.read_excel('BankRec_BankofAmerica - Copy.xlsx')

# Quick preview of the data frame
tranx.head()

,CheckCleared,DateCleared,AmountCleared
0,1,2011-06-29,222.1492
1,2,2011-06-29,300.0000
2,3,2011-06-29,9776.2665
3,4,2011-06-29,189.0395
4,5,2011-07-13,22539.0165


In [9]:
# Create a new dta frame with combining the previous two data frames
review = pd.merge(vendor, tranx, how='outer', left_on='PurchaseOrderID', right_on='CheckCleared', indicator='location')

# Quick preview of the data frame
review.head()

,PurchaseOrderID,RevisionNumber,Status,EmployeeID,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate,CheckCleared,DateCleared,AmountCleared,location
0,1.0,4.0,4.0,258.0,1580.0,3.0,2011-04-16,2011-04-25,201.0400,16.0832,5.0260,222.1492,2011-04-25,1.0,2011-06-29,222.1492,both
1,2.0,4.0,1.0,254.0,1496.0,5.0,2011-04-16,2011-04-25,272.1015,21.7681,6.8025,300.6721,2011-04-25,2.0,2011-06-29,300.0000,both
2,3.0,4.0,4.0,257.0,1494.0,2.0,2011-04-16,2011-04-25,8847.3000,707.7840,221.1825,9776.2665,2011-04-25,3.0,2011-06-29,9776.2665,both
3,4.0,4.0,3.0,261.0,1650.0,5.0,2011-04-16,2011-04-25,171.0765,13.6861,4.2769,189.0395,2011-04-25,4.0,2011-06-29,189.0395,both
4,5.0,4.0,4.0,251.0,1654.0,4.0,2011-04-30,2011-05-09,20397.3000,1631.7840,509.9325,22539.0165,2011-05-09,5.0,2011-07-13,22539.0165,both


In [10]:
# 'unknown' will be used to indicate transactions that do not exist in the check register
unknown = review.loc[review['location'] == 'right_only']

# A quick preview
unknown.head()

,PurchaseOrderID,RevisionNumber,Status,EmployeeID,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate,CheckCleared,DateCleared,AmountCleared,location
4012,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,10013.0,2012-02-26,2032.6535,right_only
4013,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,10027.0,2012-02-27,798.4841,right_only


In [11]:
# 'known' will be used to indicate transactions that do exist in the check register
known = review.loc[review['location'] == 'both']

# I will also add a column that will indicate if any difference exists between the actual check and the transaction
# amount from the bank
known['Difference'] = known['TotalDue'] - known['AmountCleared']

# A quick preview
known.head()

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,PurchaseOrderID,RevisionNumber,Status,EmployeeID,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate,CheckCleared,DateCleared,AmountCleared,location,Difference
0,1.0,4.0,4.0,258.0,1580.0,3.0,2011-04-16,2011-04-25,201.0400,16.0832,5.0260,222.1492,2011-04-25,1.0,2011-06-29,222.1492,both,0.0000
1,2.0,4.0,1.0,254.0,1496.0,5.0,2011-04-16,2011-04-25,272.1015,21.7681,6.8025,300.6721,2011-04-25,2.0,2011-06-29,300.0000,both,0.6721
2,3.0,4.0,4.0,257.0,1494.0,2.0,2011-04-16,2011-04-25,8847.3000,707.7840,221.1825,9776.2665,2011-04-25,3.0,2011-06-29,9776.2665,both,0.0000
3,4.0,4.0,3.0,261.0,1650.0,5.0,2011-04-16,2011-04-25,171.0765,13.6861,4.2769,189.0395,2011-04-25,4.0,2011-06-29,189.0395,both,0.0000
4,5.0,4.0,4.0,251.0,1654.0,4.0,2011-04-30,2011-05-09,20397.3000,1631.7840,509.9325,22539.0165,2011-05-09,5.0,2011-07-13,22539.0165,both,0.0000


In [12]:
# Now I will create another data frame to show only checks that cleared with a different amount
DiffAmount = known.loc[known['Difference'] != 0]

DiffAmount

,PurchaseOrderID,RevisionNumber,Status,EmployeeID,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate,CheckCleared,DateCleared,AmountCleared,location,Difference
1,2.0,4.0,1.0,254.0,1496.0,5.0,2011-04-16,2011-04-25,272.1015,21.7681,6.8025,300.6721,2011-04-25,2.0,2011-06-29,300.0,both,0.6721


In [13]:
# repeated code for visual reference.
unknown

,PurchaseOrderID,RevisionNumber,Status,EmployeeID,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate,CheckCleared,DateCleared,AmountCleared,location
4012,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,10013.0,2012-02-26,2032.6535,right_only
4013,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,10027.0,2012-02-27,798.4841,right_only
